this series of tutorials can be found at:

https://www.analyticsvidhya.com/blog/2016/09/comprehensive-introduction-to-apache-spark-rdds-dataframes-using-pyspark/

https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/

https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/

to use jupyter notebook with pyspark: https://blog.sicara.com/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f

In [1]:
import findspark
findspark.init()
import pyspark
import random

creating sparkcontext

In [2]:
from pyspark import SparkContext
sc = SparkContext()

In [34]:
sc.version

'2.2.0'

creating a rdd

In [3]:
data = range(1,1000)
rdd = sc.parallelize(data)

In [4]:
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [5]:
rdd.take(2) # It will print first 2 elements of rdd

[1, 2]

Step 3: Map transformation.

In [6]:
data = ['Hello' , 'I' , 'AM', 'Ankit ', 'Gupta']
Rdd = sc.parallelize(data)

the map transformation creates a new rdd:

In [7]:
Rdd1 = Rdd.map(lambda x: (x,1))

In [8]:
Rdd1.collect()


[('Hello', 1), ('I', 1), ('AM', 1), ('Ankit ', 1), ('Gupta', 1)]

If you noticed, nothing happened after applying the lambda function on Rdd1 (we won’t see any computation happening in a cluster). This is called the lazy operation. All transformation operations in Spark are lazy, which means that we will not see any computations on RDD, until we need them for further action.

### To load a csv file

In [5]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('../../../data/raw/transactions.csv',header=True)
# transactions = sqlContext.load(source="com.databricks.spark.csv", path = '../../../data/raw/transactions.csv', header = True,inferSchema = True)

first 10 rows

In [32]:
df.take(10)

[Row(msno='YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=', payment_method_id='41', payment_plan_days='30', plan_list_price='129', actual_amount_paid='129', is_auto_renew='1', transaction_date='20150930', membership_expire_date='20151101', is_cancel='0'),
 Row(msno='AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=', payment_method_id='41', payment_plan_days='30', plan_list_price='149', actual_amount_paid='149', is_auto_renew='1', transaction_date='20150930', membership_expire_date='20151031', is_cancel='0'),
 Row(msno='UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=', payment_method_id='41', payment_plan_days='30', plan_list_price='129', actual_amount_paid='129', is_auto_renew='1', transaction_date='20150930', membership_expire_date='20160427', is_cancel='0'),
 Row(msno='M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=', payment_method_id='39', payment_plan_days='30', plan_list_price='149', actual_amount_paid='149', is_auto_renew='1', transaction_date='20150930', membership_expire_date='20151128', 

### machine learning example

https://datahack.analyticsvidhya.com/contest/black-friday/

In [6]:
train = sqlContext.read.csv('../../../data/blackfriday/train.csv',header=True)
test = sqlContext.read.csv('../../../data/blackfriday/test.csv',header =True)

In [7]:
train.printSchema()

root
 |-- User_ID: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Product_Category_1: string (nullable = true)
 |-- Product_Category_2: string (nullable = true)
 |-- Product_Category_3: string (nullable = true)
 |-- Purchase: string (nullable = true)



In [8]:
train.head(10)

[Row(User_ID='1000001', Product_ID='P00069042', Gender='F', Age='0-17', Occupation='10', City_Category='A', Stay_In_Current_City_Years='2', Marital_Status='0', Product_Category_1='3', Product_Category_2=None, Product_Category_3=None, Purchase='8370'),
 Row(User_ID='1000001', Product_ID='P00248942', Gender='F', Age='0-17', Occupation='10', City_Category='A', Stay_In_Current_City_Years='2', Marital_Status='0', Product_Category_1='1', Product_Category_2='6', Product_Category_3='14', Purchase='15200'),
 Row(User_ID='1000001', Product_ID='P00087842', Gender='F', Age='0-17', Occupation='10', City_Category='A', Stay_In_Current_City_Years='2', Marital_Status='0', Product_Category_1='12', Product_Category_2=None, Product_Category_3=None, Purchase='1422'),
 Row(User_ID='1000001', Product_ID='P00085442', Gender='F', Age='0-17', Occupation='10', City_Category='A', Stay_In_Current_City_Years='2', Marital_Status='0', Product_Category_1='12', Product_Category_2='14', Product_Category_3=None, Purchase

number of rows

In [9]:
train.count()

550068

impute missing values

In [10]:
train.na.drop().count(),test.na.drop('any').count()

(166821, 71037)

In [11]:
train = train.fillna(-1)
test = test.fillna(-1)

summary

In [12]:
train.describe().show()

+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|       Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|            550068|    550068|550068|550068|           550068|       550068|                    550068|             550068|            550068|            376430|            166821|           550068|
|   mean|1003028.8424013031|      null|  null|  null|8.076706879876669|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 9.84232925112238

In [13]:
train.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase']

In [14]:
train.select('User_ID').show()

+-------+
|User_ID|
+-------+
|1000001|
|1000001|
|1000001|
|1000001|
|1000002|
|1000003|
|1000004|
|1000004|
|1000004|
|1000005|
|1000005|
|1000005|
|1000005|
|1000005|
|1000006|
|1000006|
|1000006|
|1000006|
|1000007|
|1000008|
+-------+
only showing top 20 rows



distribution of categorical features

In [15]:
train.select('Product_ID').distinct().count(), test.select('Product_ID').distinct().count()


(3631, 3491)

number of categories for Product_ID, which are in test but not in train by applying subtract method

In [16]:
diff_cat_in_train_test=test.select('Product_ID').subtract(train.select('Product_ID'))
diff_cat_in_train_test.distinct().count()# For distict count

46

Transforming categorical variables to labels

In [40]:
from pyspark.ml.feature import StringIndexer
plan_indexer = StringIndexer(inputCol = 'Product_ID', outputCol = 'product_ID_new')
labeller = plan_indexer.fit(train)

Above, we build a ‘labeller’ by applying fit() method on train Dataframe. Later we will use this ‘labeller’ to transform our train and test. Let us transform our train and test Dataframe with the help of labeller. We need to call transform method for doing that. We will store the transformation result in Train1 and Test1.



In [41]:
Train1 = labeller.transform(train)
Test1 = labeller.transform(test)

In [42]:
Train1.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+--------------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|product_ID_new|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+--------------+
|1000001| P00069042|     F| 0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|         766.0|
|1000001| P00248942|     F| 0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|         183.0|
|1000001| P00087842|     F| 0-17|        10|            A|                         2|     

successfully added one transformed column product_ID_new in our previous train Dataframe

### feature selection

In [43]:
df = sqlContext.createDataFrame([
    (1.0, 1.0, "a"),
    (0.0, 2.0, "b"),
    (0.0, 0.0, "a")
], ["y", "x", "s"])

In [51]:
rf = RFormula(formula="y ~ x + s")

In [52]:
rf.fit(df).transform(df).show()

+---+---+---+---------+-----+
|  y|  x|  s| features|label|
+---+---+---+---------+-----+
|1.0|1.0|  a|[1.0,1.0]|  1.0|
|0.0|2.0|  b|[2.0,0.0]|  0.0|
|0.0|0.0|  a|[0.0,1.0]|  0.0|
+---+---+---+---------+-----+



In [79]:
from pyspark.ml.feature import RFormula
formula = RFormula(formula="Purchase ~ Age + Occupation +City_Category + Stay_In_Current_City_Years + Product_Category_1 + Product_Category_2 + Gender",featuresCol="features",labelCol="label")

In [80]:
t1 = formula.fit(Train1)
train1 = t1.transform(Train1)
test1 = t1.transform(Test1)

In [81]:
train1.dtypes

[('User_ID', 'string'),
 ('Product_ID', 'string'),
 ('Gender', 'string'),
 ('Age', 'string'),
 ('Occupation', 'string'),
 ('City_Category', 'string'),
 ('Stay_In_Current_City_Years', 'string'),
 ('Marital_Status', 'string'),
 ('Product_Category_1', 'string'),
 ('Product_Category_2', 'string'),
 ('Product_Category_3', 'string'),
 ('Purchase', 'string'),
 ('product_ID_new', 'double'),
 ('features', 'vector'),
 ('label', 'double')]

something wrong with the version or whatever from here.. but dataframe transformation can be done with no problem

In [89]:
train1.select('label').show()
train1.select('features').show()

+-------+
|  label|
+-------+
|11529.0|
| 2312.0|
| 5917.0|
| 9163.0|
|  245.0|
| 2246.0|
| 5247.0|
| 3251.0|
| 3012.0|
|  774.0|
|  614.0|
| 5830.0|
| 1525.0|
| 2777.0|
|  830.0|
| 6344.0|
|11581.0|
| 1422.0|
| 3371.0|
| 4244.0|
+-------+
only showing top 20 rows



Py4JJavaError: An error occurred while calling o1365.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 211.0 failed 1 times, most recent failure: Lost task 0.0 in stage 211.0 (TID 1295, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:234)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:228)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.scala:827)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartition

### model: random forest

In [74]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor()

In [75]:
(train_cv, test_cv) = train1.randomSplit([0.7, 0.3])

In [76]:
model1 = rf.fit(train_cv)
predictions = model1.transform(test_cv)

Py4JJavaError: An error occurred while calling o891.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 183.0 failed 1 times, most recent failure: Lost task 0.0 in stage 183.0 (TID 1216, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.sort_addToSorter$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.c

In [101]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator()
mse = evaluator.evaluate(predictions,{evaluator.metricName:"mse" })
import numpy as np
np.sqrt(mse), mse

NameError: name 'predictions' is not defined

In [102]:
model = rf.fit(train1)
predictions1 = model.transform(test1)

Py4JJavaError: An error occurred while calling o1379.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 163.0 failed 1 times, most recent failure: Lost task 0.0 in stage 163.0 (TID 1656, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.

In [103]:
df = predictions1.selectExpr("User_ID as User_ID", "Product_ID as Product_ID", 'prediction as Purchase')

NameError: name 'predictions1' is not defined

In [104]:
df.toPandas().to_csv('submission.csv')

KeyboardInterrupt: 